In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.animation as animation
from matplotlib.collections import PatchCollection
import matplotlib.patches as pat
import h5py

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

%load_ext Cython

In [ ]:
# Ensures Proper Plots
stylesheet = {'axes.titlesize': 20,
              'axes.linewidth': 1.2,
              'axes.labelsize': 16,
              'font.size': 14,
              'legend.fontsize': 14,
              'xtick.labelsize': 14,
              'xtick.major.size': 5,
              'xtick.major.width': 1.2,
              'ytick.labelsize': 14,
              'ytick.major.size': 5,
              'ytick.major.width': 1.2,
              'figure.autolayout': True}
plt.rcParams.update(stylesheet)

<h2>Functions</h2>

In [ ]:
def set_FT_vec(l_a,l_s,l_n,l_Fin,l_Tin,dt=0.004,radius=1,k_constant=1):
    """
    Sets F_self, F_in, T_bound, T_noise, T_align using the dimensionless scaling parameters lambda_i.
    Args:
        l_a: float, Relative strength of alignment
        l_s: float, Relative strenght of self-propulsion
        l_n: float, Relative strength of noise
        l_Fin: float, Relative strength of inward force
        l_Tin: float, Relative strength of inward-turning torque
        dt: float, size of timestep (default = 0.004)
        radius: float, the mean radius of a particle (default = 1)
        k_constant: float, the spring constant of the repulsion force (default = 1)
    Output:
        FT_vec : [F_self, F_in, T_bound, T_noise, T_align]   
    """
    F_self = l_s*k_constant*radius
    F_in = l_Fin*k_constant*radius
    T_bound = l_Tin*k_constant
    T_noise = np.sqrt(l_n*2*k_constant/dt)
    T_align = l_a*k_constant
    return np.array([F_self, F_in, T_bound, T_noise, T_align])

In [ ]:
def initialize(W_input,L_input,sigma=0.1,radius=1):
    """
    Places particles on rectangular lattice, with small random deviations from the lattice sites.
    Orientation is along the long edge of the rectangle, with deviations of pi/4. 
    The sizes of the particles have a random deviation of 0.1*radius.
    Args:
        W: int, width of the rectangle.
        L: int, length of the rectangle.
        sigma: float, deviation of square lattice positions. (default = 0.1)
        radius: float, average radius of particles. (default = 1)
    Output:
        size: np.array(N), radius of the particles.
        r: np.array (2,N), position of the particles. 
        psi: np.array (N), orientation of the particles given as the angle they make with the x-axis [-pi, pi].
        v: np.array (2,N), velocity of the particles.
        omega: np.array (N), angular velocity of the particles.
    """
    
    W = min(W_input,L_input)
    L = max(W_input,L_input)
    N = W*L
    
    size = 0.1*radius*np.random.randn(N)+radius
    r = np.zeros((2,N))
    
    for i in range(W):
        r[0,L*i:L+L*i] = 2*radius*np.arange(L) + sigma*np.random.randn(L)  #x-position
        r[1,L*i:L+L*i] = 2*radius*i + sigma*np.random.rand(L)              #y-position
        
    v = np.zeros((2,N))
    psi = 0.5*np.pi*np.random.rand(N) - np.pi*0.25
    omega = np.zeros((N))
    
    return size, r, psi, v, omega

In [ ]:
%%cython
import cython
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport sin, cos, atan2
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
@cython.cdivision(True)

def comp_force_torque_C(np.ndarray[np.double_t, ndim=1] size,
                      np.ndarray[np.double_t, ndim=2] r,
                      np.ndarray[np.double_t, ndim=1] psi,
                      np.ndarray[np.double_t, ndim=1] FT_vec,
                      np.double_t radius=1,
                      np.double_t k_constant=1):
    """
    Computes the force and torque on the particles.
    Args:
        size: np.array(N), radius of the particles.
        r: np.array (2,N), position of the particles. 
        psi: np.array (N), orientation of the particles.
        FT_vec: np.array (5), array containing: [F_self, F_in, T_bound, T_noise, T_align]
        radius: float, average radius of particles. (default = 1)
        k_constant: float, the spring constant of the repulsion force (default = 1)
    Output:
        force: np.array (2,N), Force on each particle.
        torque: np.array (N), Torque on each particle.
        defect_storage: np.array(M), stores the indices of particles that are a defect.
        defect_r: np.array(2,M), stores the x and y coordinates of the defects.
        defect_counter: int, the number of defects.
        overlap: the overlap of all particles.
    """
    #Defines several constants: number of particles N, relative interaction strenghts, and some needed variables
    cdef int N = np.size(psi)
    cdef np.double_t F_self, F_in, T_bound, T_noise, T_align, r_abs, d, dpsi, \
    tetha_out, tetha_in, delta_tetha, total
    cdef np.double_t pi = np.pi, max_defect = 1
    F_self, F_in, T_bound, T_noise, T_align = FT_vec
    cdef np.double_t rcutoff = radius*2.7
    
    #Defines the needed arrays
    cdef np.ndarray[np.double_t, ndim=2] force = F_self*np.array([np.cos(psi), np.sin(psi)])
    cdef np.ndarray[np.double_t, ndim=1] torque = (2*np.random.rand(N)-1)*T_noise
    cdef np.ndarray[np.double_t, ndim=1] overlap = np.zeros(N)
    cdef np.ndarray[np.double_t, ndim=2] storage = np.zeros((10,2)) #storage for determining the angles between particles
    cdef np.ndarray[np.double_t, ndim=2] neighbours, tetha_jk #the neighbours of a particle and the angles between them
    cdef np.ndarray[np.double_t, ndim=1] r_ij = np.zeros(2), d_ij = np.zeros(2)
    cdef np.ndarray[np.int_t, ndim=1] defect_storage = np.zeros(N, dtype=int)
    cdef np.ndarray[np.double_t, ndim=2] defect_r = np.zeros((2,N))
    cdef float check_defect[2]    
    cdef unsigned int counter, i, j, k, index, kk, defect_counter #Some needed counters
    
    #Loop over all particles and their neighbours
    defect_counter = 0
    for i in range(N):
        counter = 0
        storage = np.zeros((10,2))  #Stores the angle between two particles i and j, and the index j.
        for j in range(N):
            r_ij[0] = r[0,j] - r[0,i]
            r_ij[1] = r[1,j] - r[1,i]
            r_abs = (r_ij[0]**2 + r_ij[1]**2)**(0.5)
            if i != j and r_abs < rcutoff:
                d = (size[i]+size[j]-r_abs)
                if d<0:
                    d = 0.0
                d_ij[0] = r_ij[0] * d/r_abs
                d_ij[1] = r_ij[1] * d/r_abs
                force[0,i] -= k_constant*d_ij[0]   #Hookean repulsion
                force[1,i] -= k_constant*d_ij[1]   #Hookean repulsion
                
                overlap[i] += d
                
                #Find the angles between the neighbour particles
                dpsi = psi[j]-psi[i]
                if dpsi > pi:
                    dpsi -= pi
                elif dpsi < -pi:
                    dpsi += pi
                torque[i] += T_align*dpsi
                
                storage[counter,0] = atan2(r_ij[1],r_ij[0])
                storage[counter,1] = j
                counter += 1
                
        neighbours = storage[0:counter,:]
        neighbours = neighbours[neighbours[:,0].argsort()]
        theta_jk = np.zeros((counter,3))
        check_defect = [0.0, 0.0]
        
        #Loops over the neigbours to determine whether particle i is a defect
        for k in range(counter):
            check_defect[0] += cos(psi[neighbours[k,1]])
            check_defect[1] += sin(psi[neighbours[k,1]])
            
            if k == counter-1:
                total = 0
                for kk in range(counter):
                    total += theta_jk[kk,0]
                theta_jk[k,0] = 2*pi - total
                theta_jk[k,1] = neighbours[k,1]
                theta_jk[k,2] = neighbours[0,1]
            else:
                theta_jk[k,0] = neighbours[k+1,0] - neighbours[k,0]
                theta_jk[k,1] = neighbours[k,1]
                theta_jk[k,2] = neighbours[k+1,1]
        
        if check_defect[0]**2 + check_defect[1]**2 < max_defect:
            defect_storage[defect_counter] = i
            defect_r[0,defect_counter] = r[0,i]
            defect_r[1,defect_counter] = r[1,i]
            defect_counter += 1
        
        #Loops over neighbours to determine whether particle i is on the boundary
        theta_out = 0
        index = 0    
        for kk in range(counter):
            if theta_jk[kk,0] > theta_out:
                theta_out = theta_jk[kk,0]
                index = <int>theta_jk[kk,1]
        
        if theta_out > pi:
            theta_1 = np.arctan2(r[1,index] - r[1,i],r[0,index] - r[0,i])
            theta_in = theta_1 - (2*pi - theta_out)/2
            if theta_in < -pi:
                theta_in += 2*pi
            elif theta_in > pi:
                theta_in -= 2*pi
            delta_theta = theta_in - psi[i]
            
            force[0,i] += F_in*cos(psi[i])*(theta_out-pi)
            force[1,i] += F_in*sin(psi[i])*(theta_out-pi)
            torque[i] += delta_theta*T_bound
            overlap[i] = 0
            
    return force, torque, defect_storage, defect_r, defect_counter, overlap

In [ ]:
def timestep(r, psi, FT_vec, size, dt=0.004, radius=1, k_constant=1):
    """
    Takes one timestep using a discrete approximation of the equation of motion.
    Args:
        r: np.array (2,N), position of the particles. 
        psi: np.array (N), orientation of the particles.
        FT_vec: np.array (5), array containing: [F_self, F_in, T_bound, T_noise, T_align]
        size: np.array(N), radius of the particles.
        dt: float, size of timestep (default = 0.004)
        radius: float, average radius of particles. (default = 1)
        k_constant: float, the spring constant of the repulsion force (default = 1)
    Output:
        r: np.array (2,N), position of the particles. 
        psi: np.array (N), orientation of the particles.
        defect_r: np.array(2,M), stores the x and y coordinates of the defects.
        defect_counter: int, the number of defects.
        overlap: the overlap of all particles.
    """
    force, torque, defect, defect_r, defect_counter, overlap = comp_force_torque_C(size,r,psi,FT_vec)
    
    v = force/size
    omega = torque/(size**2)
    
    r += v*dt
    psi += omega*dt
    psi = ((psi/np.pi) - np.fix((psi/np.pi)))*np.pi
    
    return r, psi, defect, defect_r, defect_counter, overlap

In [ ]:
def simulation(f,steps,L,W,l_a,l_s,l_n,l_Fin,l_Tin):
    """
    Simulates the system for a given number of steps. A dataset is created to store all the data needed for analysis.
    Args:
        f: h5py file, contains the dataset.
        steps: int, number of timesteps.
        L: int, length of the rectangle.
        W: int, width of the rectangle.
        l_a: float, Relative strength of alignment
        l_s: float, Relative strenght of self-propulsion
        l_n: float, Relative strength of noise
        l_Fin: float, Relative strength of inward force
        l_Tin: float, Relative strength of inward-turning torque
    Output:
        f: h5py file, contains the dataset.
    """
    
    N = L*W
    
    Parameters = f.create_dataset('L_W_steps_la_ls_ln_lfin_ltin', (8,1))  
    data_r = f.create_dataset('r', (2,N,1), maxshape=(2,N,None))
    data_size = f.create_dataset('size', (N,1))
    data_psi = f.create_dataset('psi', (N,1), maxshape=(N,None))
    data_r_mean = f.create_dataset('r_mean', (2,1), maxshape = (2,None))
    data_order_param = f.create_dataset('order_param', (1,1), maxshape = (None,1))
    data_overlap = f.create_dataset('overlap', (N,1), maxshape = (N,None))
    data_defect = f.create_dataset('defect', (N,1), maxshape = (N,None))
    data_defect_r = f.create_dataset('defect_r', (2,N,1), maxshape = (2,N,None))
    data_defect_counter = f.create_dataset('defect_counter', (1,1), maxshape = (None,1))
    
    Parameters[:,0] = np.array([L, W, steps, l_a, l_s, l_n, l_Fin, l_Tin])   

    FT_vec = set_FT_vec(l_a,l_s,l_n,l_Fin,l_Tin)
    size, r, psi, v, omega = initialize(L,W)

    data_r[:,:,0] = r
    data_size[:,0] = size
    data_psi[:,0] = psi
    data_r_mean[:,0] = np.mean(r,1)
    data_defect[:,0] = np.zeros(N,dtype=int)
    data_defect_r[:,:,0] = np.zeros(N)
    data_defect_counter[:,0] = 0
    data_order_param[0,0] = (1/N)*(np.sum(np.cos(psi))**2+np.sum(np.sin(psi))**2)**(0.5)
    data_overlap[:,0] = np.zeros(N)
    
    f.flush()
    progress = 0
    print('Progress (%): ')
    for t in range(steps):
        r, psi, defect, defect_r, defect_counter, overlap = timestep(r, psi, FT_vec, size)
        data_r.resize((2,N,data_r.shape[2]+1))
        data_r[:,:,-1] = r
        data_psi.resize((N,data_psi.shape[1]+1))
        data_psi[:,-1] = psi
        data_r_mean.resize((2,data_r_mean.shape[1]+1))
        data_r_mean[:,-1] = np.mean(r,1)
        data_order_param.resize((data_order_param.shape[0]+1,1))
        data_order_param[-1] = (1/N)*(np.sum(np.cos(psi))**2+np.sum(np.sin(psi))**2)**(0.5)
        data_defect.resize((N,data_defect.shape[1]+1))
        data_defect[:,-1] = defect
        data_defect_r.resize((2,N,data_defect_r.shape[2]+1))
        data_defect_r[:,:,-1] = defect_r
        data_defect_counter.resize((data_defect_counter.shape[0]+1,1))
        data_defect_counter[-1] = defect_counter
        data_overlap.resize((N,data_overlap.shape[1]+1))
        data_overlap[:,-1] = overlap
        f.flush()
        
        if ((100*t)/steps) % 1 == 0:
            progress += 1
            print(progress, end=' ')
    return

<h2>Simple simulation</h2>

In [ ]:
L = 20
W = 10
N = W*L

l_a, l_s, l_n, l_Fin, l_Tin = np.array([0.2, 0.06, 0.03, 0.3, 3])

steps = 100

In [ ]:
f = h5py.File('.\Data\Simulation_L=%i_W=%i_steps=%i_la=%.2f_ls=%.2f.hdf5' %(L,W,steps,l_a,l_s), 'w' )
simulation(f,steps,L,W,l_a,l_s,l_n,l_Fin,l_Tin)
f.close()